In [1]:
import re
import os
import pickle
import numpy as np
import logging
import torch
import math
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel
from utils.MIND import MIND

logger = logging.getLogger(__name__)

In [2]:
# config.reducer = 'bm25'
config.reducer = 'bow'
# config.reducer = 'matching'

# config.scale = 'large'
path = config.path + 'MIND/MINDdemo_dev/'
a = MIND(config, path + 'news.tsv', path + 'behaviors.tsv')

[2021-08-27 12:23:41,387] INFO (utils.MIND) loading cached user behavior from data/cache/bert/MINDdemo_dev/10/behaviors..pkl
[2021-08-27 12:23:41,393] INFO (utils.MIND) loading cached news tokenization from data/cache/bert/MINDdemo_dev/news.pkl
[2021-08-27 12:23:41,614] INFO (utils.MIND) reducing news of ../../../Data/MIND/MINDdemo_dev/news.tsv...
[2021-08-27 12:23:41,615] INFO (utils.utils) reducing to Bag-of-Words...
[2021-08-27 12:23:43,826] INFO (utils.utils) unmasking at least k...


In [3]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False)
records1 = list(loader1)

In [7]:
[v.shape for k,v in records1[0].items()]

[torch.Size([1]),
 torch.Size([1, 1]),
 torch.Size([1, 10]),
 torch.Size([1, 50]),
 torch.Size([1, 10, 100]),
 torch.Size([1, 50, 100]),
 torch.Size([1, 10, 100]),
 torch.Size([1, 50, 100]),
 torch.Size([1, 50]),
 torch.Size([1, 10]),
 torch.Size([1, 50, 100, 2]),
 torch.Size([1, 50, 100])]

In [9]:
t = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
t.decode([1011])

'-'

In [4]:
records1[0]['his_reduced_index'].shape, records1[0]['his_attn_mask'][0][0] 

(torch.Size([1, 50, 100, 2]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]))

In [15]:
t = BertTokenizer.from_pretrained('bert-base-uncased')

In [17]:
t.decode(list(c[1].keys()))

"prince swear philip jackets notebook brands royals queen charlesals elizabethroy shop live without lifestyle can more t by thats and '., the"